In [20]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [21]:
vil = pd.read_csv("/Users/sid/Downloads/Raj_vil_ma.csv")
vil = vil.drop(['Unnamed: 0'],axis = 1)

## Dropping villages with NaN population
vil = vil.dropna(subset=['Total Population'])

gdf1 = gpd.read_file('/Users/sid/Documents/maps-master/geojson/raj_panchayat.geojson')
gdf2 = gpd.read_file('/Users/sid/Documents/maps-master/geojson/raj_ac.geojson')
gdf3 = gpd.read_file('/Users/sid/Documents/maps-master/geojson/raj_dist.geojson')
# gdf4 = gpd.read_file('raj_dist.geojson')
# vil.columns

In [4]:

## Set GP as index to run weighted average
# df1 = vil.set_index('Gram Panchayat')
# df1.groupby(df1.index).apply(lambda x: np.average(x.wt, weights=x.value))
## Works only if sum!= 0 otherwise create a function with try and except

## Function for weighted avg on one factor
# def agri(x):
#     try:
#         #return np.average(x, weights=df1.loc[x.index, 'Agriculture'])
#         return np.average(x.Agriculture, weights=x.Population)
#     except ZeroDivisionError:
#         return 0

# ## Group by GP 
# agri = vil.groupby(['Gram Panchayat']).apply(agri)  

# ## Convert series to df
# agri = agri.to_frame("Agriculture").reset_index()

In [5]:

## Function for weighted avg on multiple factors
def wt_avg(x, cols, w="Total Population"):
    try:
        return pd.Series(np.average(x[cols], weights = x[w], axis =0), cols)
    except ZeroDivisionError:
        return 0
    
# df.apply(weighted, ["var1", "var2"])
agg = vil.groupby(["Gram Panchayat"]).apply(wt_avg,['Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Welfare of the Weaker Sections', 'Poverty Alleviation Programme (0-6)',
       'Khadi, Village & Cottage Industries (0-3)', 'Social Forestry (0-1).1',
       'Minor Forest Produce', 'Small Scale Industries (0-1)',
       'Adult & Non-Formal Education (0-1)']).reset_index()

## Merge new df with old to get AC names
vill = agg.merge(vil[['District', 'Sub District', 'Development Block', 'Parliament Constituency', 
                      'Assembly Constituency','Gram Panchayat','District Code', 'Sub District Code', 
                      'Block Code','GP Code', 'Vil Code', 'PC Code', 
                      'Assembly Constituency Code', 'AC Code']], 
                how = 'left', on ='Gram Panchayat')

## Drop duplicates after Merge
vill = vill.drop_duplicates(subset=['Gram Panchayat'])

## summing population over GP
pop = vil.groupby('Gram Panchayat')[["Total Population"]].sum().reset_index()

## Merge pop. to df
vill = vill.merge(pop[['Gram Panchayat','Total Population']],
                 how = 'left', on = 'Gram Panchayat')

## splitting GP code and creating a column
## Due to uneven number of str, run twice 
vill["Code1"] = vill["Gram Panchayat"].str.split('(').str[1]
vill["GP Code"] = vill["Gram Panchayat"].str.split('(').str[2]
# vill["Code1"] = vill["Code1"].str.strip(')')


## Fill na values in GP Code
vill['GP Code'] = vill['GP Code'].fillna(vill['Code1'])
vill["GP Code"] = vill["GP Code"].str.strip(')')

## Change dtype
vill['GP Code'] = vill['GP Code'].astype(float)

## Drop the unnecesary column
vill = vill.drop(['Code1'],axis = 1)

## Export to csv
# vill.to_csv("GP_ma.csv")

In [18]:
vil_ma = pd.read_csv('Raj_GP_ma.csv')
vil_ma.columns

Index(['District', 'District Code', 'Sub District', 'Sub District Code',
       'Development Block', 'Block Code', 'Parliament Constituency', 'AC_NAME',
       'AC Code', 'GP Code', 'Total Population', 'Gram Panchayat',
       'Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Welfare of the Weaker Sections', 'Poverty Alleviation Programme (0-6)',


In [45]:

## AC weighted averages
agg1 = vil_ma.groupby(["AC Code"]).apply(wt_avg,['Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Welfare of the Weaker Sections', 'Poverty Alleviation Programme (0-6)',
       'Khadi, Village & Cottage Industries (0-3)', 'Social Forestry (0-1)',
       'Minor Forest Produce', 'Small Scale Industries (0-1).1',
       'Adult & Non-Formal Education (0-1)']).reset_index()

## Pop. groupby
pop1 = vil_ma.groupby('AC Code')[["Total Population"]].sum().reset_index()

## Merge pop. to agg1
agg1 = agg1.merge(pop1[['AC Code','Total Population']],
                 how = 'left', on = 'AC Code')

# agg1 = agg1.merge(vil_ma[['AC_NAME','AC Code','District']],
#                  how = 'left', on = 'AC Code')

## Merge AC & District name
agg1 = agg1.merge(gdf2[["AC_NAME","AC_NO","DIST_NAME"]],
                 how = 'left', left_on = 'AC Code', right_on = 'AC_NO')

# agg1.to_csv("Raj_AC_ma.csv")

In [46]:

## District weighted avgs
agg2 = vil.groupby(["District"]).apply(wt_avg,['Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Welfare of the Weaker Sections', 'Poverty Alleviation Programme (0-6)',
       'Khadi, Village & Cottage Industries (0-3)', 'Social Forestry (0-1).1',
       'Minor Forest Produce', 'Small Scale Industries (0-1)',
       'Adult & Non-Formal Education (0-1)']).reset_index()

# agg2.to_csv("Raj_dt_ma.csv")

In [22]:
dtma = pd.read_csv('Raj_dt_ma.csv')
acma = pd.read_csv('Raj_AC_ma.csv')

In [180]:
dt = gdf4.merge(dtma, how = 'left',
               left_on = 'DISTRICT',
               right_on = 'District')

dt

,DISTRICT,ST_NM,ST_CEN_CD,DT_CEN_CD,censuscode,geometry,District,Agriculture (0-7),Land Improvement & Minor Irrigation (0-9),Animal Husbandry (0-6),...,Markets & Fairs (0-3),Health & Sanitation (0-11),Women & Child Development (0-7),Social Forestry (0-1),Family Welfare (0-1),Poverty Alleviation Programme (0-6),"Khadi, Village & Cottage Industries (0-3)",Social Forestry (0-1).1,Small Scale Industries (0-1),Adult & Non-Formal Education (0-1)
0,AJMER,Rajasthan,8,21,119,"MULTIPOLYGON (((74.89253 26.97702, 74.90465 26...",AJMER,1.51,2.33,2.59,...,1.12,4.10,2.23,0.24,0.50,0.54,0.01,0.24,0.20,0.06
1,ALWAR,Rajasthan,8,6,104,"MULTIPOLYGON (((76.86341 28.22005, 76.86680 28...",ALWAR,1.44,3.54,1.78,...,0.83,4.30,3.29,0.15,0.47,0.43,0.07,0.15,0.12,0.16
2,BANSWARA,Rajasthan,8,27,125,"POLYGON ((74.49053 23.90931, 74.49327 23.90164...",BANSWARA,1.81,3.63,2.49,...,1.35,3.47,3.06,0.10,0.44,1.30,0.07,0.10,0.03,0.08
3,BARAN,Rajasthan,8,30,128,"POLYGON ((76.71961 25.33754, 76.72190 25.33396...",BARAN,1.67,3.44,2.35,...,1.60,4.40,3.76,0.08,0.57,0.99,0.04,0.08,0.03,0.00
4,BARMER,Rajasthan,8,17,115,"POLYGON ((72.07268 26.34943, 72.08514 26.33629...",BARMER,0.97,1.89,0.98,...,0.59,3.66,2.52,0.22,0.30,0.37,0.10,0.22,0.00,0.03
5,BHARATPUR,Rajasthan,8,7,105,"POLYGON ((77.28461 27.80099, 77.28815 27.79723...",BHARATPUR,1.95,3.25,1.73,...,1.73,4.32,2.74,0.17,0.50,0.60,0.17,0.17,0.09,0.17
6,BHILWARA,Rajasthan,8,24,122,"POLYGON ((75.31911 25.76375, 75.36485 25.75774...",BHILWARA,1.20,3.08,1.63,...,0.76,4.04,3.04,0.19,0.48,0.60,0.11,0.19,0.21,0.09
7,BIKANER,Rajasthan,8,3,101,"POLYGON ((73.78488 29.04108, 73.78645 29.03979...",BIKANER,1.79,3.04,1.80,...,1.19,4.45,2.35,0.27,0.56,0.54,0.00,0.27,0.24,0.19
8,BUNDI,Rajasthan,8,23,121,"POLYGON ((76.15723 25.76174, 76.15010 25.75469...",BUNDI,1.75,4.50,1.58,...,1.17,4.07,3.78,0.05,0.50,0.83,0.00,0.05,0.01,0.07
9,CHITTAURGARH,Rajasthan,8,28,126,"MULTIPOLYGON (((74.78757 25.18170, 74.78529 25...",CHITTAURGARH,2.07,3.56,2.00,...,1.01,4.69,3.52,0.26,0.65,0.76,0.00,0.26,0.09,0.09


In [181]:
dt.to_file('Raj_dt_ma.geojson', driver = 'GeoJSON')

In [19]:
ac = gdf2.merge(acma, how = 'left',
               left_on = 'AC_NO',
               right_on = 'AC Code')

ac['DIST_NAME'].unique()

array(['GANGANAGAR', 'HANUMANGARH', 'BIKANER', 'CHURU', 'JHUNJHUNU',
       'JAISALMER', 'SIKAR', 'ALWAR', 'JAIPUR', 'JODHPUR', 'BHARATPUR',
       'NAGAUR', 'DAUSA', 'AJMER', 'KARAULI', 'DHAULPUR', 'BARMER',
       'SAWAI MADHOPUR', 'TONK', 'PALI', 'BHILWARA', 'JALOR', 'RAJSAMAND',
       'BUNDI', 'KOTA', 'SIROHI', 'BARAN', 'CHITTAURGARH', 'UDAIPUR',
       'JHALAWAR', 'PRATAPGARH', 'DUNGARPUR', 'BANSWARA'], dtype=object)

In [18]:
ac.to_file('Raj_AC_ma.geojson', driver = 'GeoJSON')

In [15]:
gdf2.columns

Index(['ST_CODE', 'ST_NAME', 'DT_CODE', 'DIST_NAME', 'AC_NO', 'AC_NAME',
       'PC_NO', 'PC_NAME', 'geometry'],
      dtype='object')

In [17]:
acma

,District,Assembly Constituency,AC Code,Total Population,Agriculture (0-7),Land Improvement & Minor Irrigation (0-9),Animal Husbandry (0-6),Fisheries (0-2),Rural Housing (0-4),Drinking Water (0-3),...,Markets & Fairs (0-3),Health & Sanitation (0-11),Women & Child Development (0-7),Social Forestry (0-1),Family Welfare (0-1),Poverty Alleviation Programme (0-6),"Khadi, Village & Cottage Industries (0-3)",Social Forestry (0-1).1,Small Scale Industries (0-1),Adult & Non-Formal Education (0-1)
0,GANGANAGAR,Sadulshahar,1,13749,0.72,4.63,1.36,0.67,3.99,2.39,...,0.20,5.00,4.05,0.00,0.72,0.31,0.00,0.00,0.00,0.51
1,GANGANAGAR,Ganganagar,2,10240,1.38,3.52,2.35,0.00,3.82,2.28,...,0.62,3.84,2.72,0.00,0.71,1.32,0.00,0.00,0.00,0.00
2,GANGANAGAR,Karanpur,3,12074,1.07,5.24,0.85,0.00,3.58,2.03,...,1.01,3.32,3.68,0.00,0.46,0.53,0.07,0.00,0.03,0.00
3,GANGANAGAR,Suratgarh,4,16780,1.12,4.13,1.16,0.00,3.12,2.15,...,0.34,2.52,2.68,0.19,0.38,0.08,0.09,0.19,0.10,0.14
4,GANGANAGAR,Raisinghnagar,5,29491,0.87,4.56,0.66,0.00,3.33,1.93,...,0.31,3.18,3.31,0.03,0.51,0.35,0.03,0.03,0.00,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,BARAN,Chhabra,196,19624,1.21,2.54,0.70,0.37,3.56,0.49,...,0.53,3.81,3.42,0.17,0.27,1.39,0.00,0.17,0.01,0.00
186,JHALAWAR,Dag,197,43661,1.40,2.89,0.76,0.09,2.54,1.84,...,0.62,3.95,3.14,0.12,0.54,0.55,0.00,0.12,0.08,0.10
187,JHALAWAR,Jhalrapatan,198,15600,3.17,3.29,2.93,0.54,3.45,1.94,...,1.03,5.19,3.36,0.62,0.74,1.64,0.29,0.62,0.00,0.20
188,JHALAWAR,Khanpur,199,22003,2.54,4.00,2.14,0.00,3.23,2.34,...,1.73,4.14,2.18,0.47,0.55,1.17,0.02,0.47,0.00,0.00


In [203]:
dt.to_file('dt.geojson')

In [23]:
ff = gpd.read_file('/Users/sid/Documents/Jupyter/testfile/Raj_dt_ma.geojson')

In [25]:
fff = gpd.read_file("dt.geojson")
fff

,DISTRICT,ST_NM,ST_CEN_CD,DT_CEN_CD,censuscode,Agriculture (0-7),Land Improvement & Minor Irrigation (0-9),Animal Husbandry (0-6),Fisheries (0-2),Rural Housing (0-4),...,Health & Sanitation (0-11),Women & Child Development (0-7),Social Forestry (0-1),Family Welfare (0-1),Poverty Alleviation Programme (0-6),"Khadi, Village & Cottage Industries (0-3)",Social Forestry (0-1).1,Small Scale Industries (0-1),Adult & Non-Formal Education (0-1),geometry
0,AJMER,Rajasthan,8,21,119,1.51,2.33,2.59,0.10,3.93,...,4.10,2.23,0.24,0.50,0.54,0.01,0.24,0.20,0.06,"MULTIPOLYGON (((74.89253 26.97702, 74.90465 26..."
1,ALWAR,Rajasthan,8,6,104,1.44,3.54,1.78,0.05,3.98,...,4.30,3.29,0.15,0.47,0.43,0.07,0.15,0.12,0.16,"MULTIPOLYGON (((76.86341 28.22005, 76.86680 28..."
2,BANSWARA,Rajasthan,8,27,125,1.81,3.63,2.49,0.08,3.11,...,3.47,3.06,0.10,0.44,1.30,0.07,0.10,0.03,0.08,"POLYGON ((74.49053 23.90931, 74.49327 23.90164..."
3,BARAN,Rajasthan,8,30,128,1.67,3.44,2.35,0.12,3.19,...,4.40,3.76,0.08,0.57,0.99,0.04,0.08,0.03,0.00,"POLYGON ((76.71961 25.33754, 76.72190 25.33396..."
4,BARMER,Rajasthan,8,17,115,0.97,1.89,0.98,0.00,3.06,...,3.66,2.52,0.22,0.30,0.37,0.10,0.22,0.00,0.03,"POLYGON ((72.07268 26.34943, 72.08514 26.33629..."
5,BHARATPUR,Rajasthan,8,7,105,1.95,3.25,1.73,0.08,3.97,...,4.32,2.74,0.17,0.50,0.60,0.17,0.17,0.09,0.17,"POLYGON ((77.28461 27.80099, 77.28815 27.79723..."
6,BHILWARA,Rajasthan,8,24,122,1.20,3.08,1.63,0.15,3.75,...,4.04,3.04,0.19,0.48,0.60,0.11,0.19,0.21,0.09,"POLYGON ((75.31911 25.76375, 75.36485 25.75774..."
7,BIKANER,Rajasthan,8,3,101,1.79,3.04,1.80,0.05,3.36,...,4.45,2.35,0.27,0.56,0.54,0.00,0.27,0.24,0.19,"POLYGON ((73.78488 29.04108, 73.78645 29.03979..."
8,BUNDI,Rajasthan,8,23,121,1.75,4.50,1.58,0.02,3.26,...,4.07,3.78,0.05,0.50,0.83,0.00,0.05,0.01,0.07,"POLYGON ((76.15723 25.76174, 76.15010 25.75469..."
9,CHITTAURGARH,Rajasthan,8,28,126,2.07,3.56,2.00,0.02,3.85,...,4.69,3.52,0.26,0.65,0.76,0.00,0.26,0.09,0.09,"MULTIPOLYGON (((74.78757 25.18170, 74.78529 25..."


In [26]:
fff.columns

Index(['DISTRICT', 'ST_NM', 'ST_CEN_CD', 'DT_CEN_CD', 'censuscode',
       'Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Poverty Alleviation Programme (0-6)',
       'Khadi, Village & Cottage Industries (0-3)', 'Social Forestry (0-1).1',
       'Small Scale Industries (0-1)', 'Adult & Non-Formal Education (0-1)',
       'geometry'],
      dtyp

In [198]:
acma1 = pd.read_csv('/Users/sid/Documents/Jupyter/testfile/Raj_AC_ma.csv')

In [199]:
acma1.columns

Index(['District', 'Assembly Constituency', 'AC Code', 'Total Population',
       'Agriculture (0-7)', 'Land Improvement & Minor Irrigation (0-9)',
       'Animal Husbandry (0-6)', 'Fisheries (0-2)', 'Rural Housing (0-4)',
       'Drinking Water (0-3)', 'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Poverty Alleviation Programme (0-6)',
       'Khadi, Village & Cottage Industries (0-3)', 'Social Forestry (0-1).1',
       'Small Scale Industries (0-1)', 'Adult & Non-Formal Education (0-1)'],
      dtype='object')

In [202]:
dt['District']

0              AJMER
1              ALWAR
2           BANSWARA
3              BARAN
4             BARMER
5          BHARATPUR
6           BHILWARA
7            BIKANER
8              BUNDI
9       CHITTAURGARH
10             CHURU
11          DHAULPUR
12         DUNGARPUR
13        GANGANAGAR
14       HANUMANGARH
15            JAIPUR
16         JAISALMER
17             JALOR
18          JHALAWAR
19               NaN
20           JODHPUR
21              KOTA
22            NAGAUR
23              PALI
24        PRATAPGARH
25         RAJSAMAND
26             SIKAR
27            SIROHI
28              TONK
29           UDAIPUR
30             DAUSA
31           KARAULI
32    SAWAI MADHOPUR
Name: District, dtype: object

In [9]:
# gdf2 = gdf2.replace(to_replace='JHUNJHUNUN',value='JHUNJHUNU')

In [35]:
gdf2['AC_NO'].desc

AttributeError: 'Series' object has no attribute 'desc'

In [36]:
# gdf2.to_file('/Users/sid/Documents/maps-master/geojson/raj_ac.geojson')

In [28]:
gdf2

,ST_CODE,ST_NAME,DT_CODE,DIST_NAME,AC_NO,AC_NAME,PC_NO,PC_NAME,geometry
0,8,RAJASTHAN,1,GANGANAGAR,1,Sadulshahar,1,GANGANAGAR(SC),"POLYGON ((73.97266 30.19826, 73.96740 30.14094..."
1,8,RAJASTHAN,1,GANGANAGAR,3,Karanpur,1,GANGANAGAR(SC),"POLYGON ((73.75147 29.78649, 73.78777 29.78573..."
2,8,RAJASTHAN,1,GANGANAGAR,2,Ganganagar,1,GANGANAGAR(SC),"POLYGON ((73.75147 29.78649, 73.74990 29.79464..."
3,8,RAJASTHAN,2,HANUMANGARH,7,Sangaria,1,GANGANAGAR(SC),"MULTIPOLYGON (((74.69479 29.36590, 74.71948 29..."
4,8,RAJASTHAN,2,HANUMANGARH,8,Hanumangarh,1,GANGANAGAR(SC),"POLYGON ((74.26456 29.80719, 74.27751 29.80469..."
...,...,...,...,...,...,...,...,...,...
195,8,RAJASTHAN,12,JAIPUR,54,Malviya Nagar,7,JAIPUR,"POLYGON ((75.83094 26.92957, 75.82165 26.92461..."
196,8,RAJASTHAN,12,JAIPUR,52,Kishanpole,7,JAIPUR,"POLYGON ((75.82826 26.94569, 75.81738 26.94754..."
197,8,RAJASTHAN,12,JAIPUR,53,Adarsh Nagar,7,JAIPUR,"POLYGON ((75.84192 26.94564, 75.86634 26.94095..."
198,8,RAJASTHAN,12,JAIPUR,49,Hawa Mahal,7,JAIPUR,"POLYGON ((75.81386 26.95871, 75.81626 26.96322..."


In [49]:
df = pd.read_csv('/Users/sid/Documents/Jupyter/testfile/Raj_GP_ma.csv')

In [50]:
df

,District,District Code,Sub District,Sub District Code,Development Block,Block Code,Parliament Constituency,Assembly Constituency,AC Code,GP Code,...,Markets & Fairs (0-3),Health & Sanitation (0-11),Women & Child Development (0-7),Social Forestry (0-1),Family Welfare (0-1),Poverty Alleviation Programme (0-6),"Khadi, Village & Cottage Industries (0-3)",Social Forestry (0-1).1,Small Scale Industries (0-1),Adult & Non-Formal Education (0-1)
0,GANGANAGAR,100,Sadulshahar,459,SADULSHAHAR,679,Ganganagar,Sadulshahar,1.0,37903.0,...,0.26,2.26,2.58,0.00,0.00,0.52,0.0,0.00,0.0,0.0
1,GANGANAGAR,100,Ganganagar,458,GANGANAGAR,675,Ganganagar,Sadulshahar,1.0,37745.0,...,1.00,3.00,2.00,0.00,1.00,2.00,0.0,0.00,0.0,0.0
2,GANGANAGAR,100,Sadulshahar,459,SADULSHAHAR,679,Ganganagar,Sadulshahar,1.0,37898.0,...,1.00,1.50,3.00,0.00,1.00,0.00,0.0,0.00,0.0,0.0
3,HANUMANGARH,101,Tibi,467,TIBBI,6616,Ganganagar,Sangaria,7.0,38015.0,...,0.50,2.50,0.50,0.00,0.00,0.00,0.0,0.00,0.0,0.0
4,BIKANER,93,Chhatargarh,479,KHAJUWALA,6715,Bikaner,Khajuwala,12.0,294057.0,...,0.00,1.83,1.18,0.00,0.00,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288,PRATAPGARH,629,Peepalkhoont,697,SUHAGPURA,7468,Chittorgarh,Pratapgarh,172.0,36761.0,...,0.90,3.32,2.53,0.75,0.61,0.11,0.0,0.75,0.0,0.0
3289,BARMER,90,DHORIMANNA,6050,DHORIMANNA,607,Barmer,NaN,NaN,262795.0,...,0.00,0.50,0.50,0.00,0.00,0.00,0.0,0.00,0.0,0.0
3290,JHUNJHUNU,106,Khetri,490,KHETRI,717,Jhunjhunu,Khetri,31.0,39496.0,...,0.00,6.00,1.00,0.17,0.17,1.00,0.0,0.17,0.0,1.0
3291,ALWAR,87,Lachhmangarh,503,KATHUMAR,576,Bharatpur,Kathumar,69.0,33968.0,...,0.50,7.00,2.00,1.00,1.00,0.00,0.0,1.00,1.0,0.0


In [55]:
gp = gdf1.merge(df, how='right',left_on='gp_code',right_on='GP Code')

In [52]:
df.columns

Index(['District', 'District Code', 'Sub District', 'Sub District Code',
       'Development Block', 'Block Code', 'Parliament Constituency',
       'Assembly Constituency', 'AC Code', 'GP Code', 'Total Population',
       'Gram Panchayat', 'Agriculture (0-7)',
       'Land Improvement & Minor Irrigation (0-9)', 'Animal Husbandry (0-6)',
       'Fisheries (0-2)', 'Rural Housing (0-4)', 'Drinking Water (0-3)',
       'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocational Education (0-2)', 'Markets & Fairs (0-3)',
       'Health & Sanitation (0-11)', 'Women & Child Development (0-7)',
       'Social Forestry (0-1)', 'Family Welfare (0-1)',
       'Poverty Alleviation Programme (0-6)',
       'Khadi

In [61]:
gp.dropna(subset=['dtname'])

,dtname,stname,OBJECTID_1,objectid,vilnam_soi,stcode11,dtcode11,sdtcode11,vilcode11,sdtname,...,Markets & Fairs (0-3),Health & Sanitation (0-11),Women & Child Development (0-7),Social Forestry (0-1),Family Welfare (0-1),Poverty Alleviation Programme (0-6),"Khadi, Village & Cottage Industries (0-3)",Social Forestry (0-1).1,Small Scale Industries (0-1),Adult & Non-Formal Education (0-1)
0,Hanumangarh,RAJASTHAN,203985.0,532321.0,15 Spm,8.0,100.0,469.0,64810.0,Pilibanga,...,0.26,2.26,2.58,0.00,0.00,0.52,0.0,0.00,0.0,0.0
1,Ganganagar,RAJASTHAN,206531.0,537655.0,9 Z,8.0,99.0,458.0,64463.0,Ganganagar,...,1.00,3.00,2.00,0.00,1.00,2.00,0.0,0.00,0.0,0.0
2,Ganganagar,RAJASTHAN,206336.0,537296.0,4 Bnw (banwala),8.0,99.0,459.0,64651.0,Sadulshahar,...,1.00,1.50,3.00,0.00,1.00,0.00,0.0,0.00,0.0,0.0
3,Hanumangarh,RAJASTHAN,201176.0,525955.0,2 Ksp,8.0,100.0,467.0,67440.0,Tibi,...,0.50,2.50,0.50,0.00,0.00,0.00,0.0,0.00,0.0,0.0
5,Ganganagar,RAJASTHAN,206669.0,537939.0,10 Bgs,8.0,99.0,459.0,64640.0,Sadulshahar,...,0.50,6.50,3.00,0.00,1.00,1.00,0.0,0.00,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3287,Alwar,RAJASTHAN,175593.0,473200.0,Titpuri,8.0,104.0,504.0,73740.0,Kathumar,...,0.50,1.00,3.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0
3288,Pratapgarh,RAJASTHAN,108357.0,261211.0,Veerpur,8.0,131.0,697.0,108196.0,Peepalkhoont,...,0.90,3.32,2.53,0.75,0.61,0.11,0.0,0.75,0.0,0.0
3289,Barmer,RAJASTHAN,134280.0,364713.0,Meghwalon Ka Tala,8.0,115.0,581.0,87654.0,Gudha Malani,...,0.00,0.50,0.50,0.00,0.00,0.00,0.0,0.00,0.0,0.0
3290,Jhunjhunun,RAJASTHAN,185217.0,493350.0,Tyonda,8.0,103.0,490.0,71526.0,Khetri,...,0.00,6.00,1.00,0.17,0.17,1.00,0.0,0.17,0.0,1.0


In [62]:
gp.columns

Index(['dtname', 'stname', 'OBJECTID_1', 'objectid', 'vilnam_soi', 'stcode11',
       'dtcode11', 'sdtcode11', 'vilcode11', 'sdtname', 'vilname', 'vilcode',
       'gp_code', 'gp_name', 'ishqtr', 'LOC_STAT_1', 'LOC_NAME', 'geometry',
       'District', 'District Code', 'Sub District', 'Sub District Code',
       'Development Block', 'Block Code', 'Parliament Constituency',
       'Assembly Constituency', 'AC Code', 'GP Code', 'Total Population',
       'Gram Panchayat', 'Agriculture (0-7)',
       'Land Improvement & Minor Irrigation (0-9)', 'Animal Husbandry (0-6)',
       'Fisheries (0-2)', 'Rural Housing (0-4)', 'Drinking Water (0-3)',
       'Roads (0-8)', 'Rural Electrification (0-3)',
       'Non-Conventional Energy (0-1)',
       'Maintenance of Community Assets (0-1)', 'Fuel & Fodder (0-1)',
       'Libraries (0-1)', 'Cultural Activities (0-1)',
       'Financial & Communication Infrastructure (0-9)',
       'Public Distribution System (0-2)', 'Education (0-6)',
       'Vocatio

In [64]:
gp.to_file('/Users/sid/Documents/Jupyter/testfile/Raj_GP_ma.geojson',driver='GeoJSON')